In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

from draft_optimizer.src.utils import DATA_DIR

# Specify paths
league_id = 88497130
year = 2022
ESPN_DIR = os.path.join(DATA_DIR, f"espn_{league_id}", str(year))
SLEEPER_DIR = os.path.join(DATA_DIR, "sleeper", str(year))
PROD_DIR = os.path.join(DATA_DIR, "production", str(year))

# Specify constants
FANTASY_POS = ["QB", "RB", "WR", "TE", "K", "D/ST"]

In [ ]:
# Load data
espn = pd.read_csv(os.path.join(ESPN_DIR, "pro_players.csv"))
sleeper_players = pd.read_csv(os.path.join(SLEEPER_DIR, "players.csv"), index_col=[0])

# Keep active players
sleeper_players = sleeper_players.loc[sleeper_players["active"]]

# Add ESPN IDs to D/ST
espn_def_ids = espn.loc[espn["position"] == "D/ST"].set_index("name")["id"]
is_def = sleeper_players["position"] == "D/ST"
sleeper_players.loc[is_def, "espn_id"] = sleeper_players.loc[is_def, "full_name"].map(espn_def_ids)

# Filter and subset
player_cols = ["full_name", "position", "team"]
players = sleeper_players.loc[sleeper_players["position"].isin(FANTASY_POS), player_cols]

# Finalize index
players.index.name = "id"
players.columns = ["name", "position", "pro_team"]

print(players.shape)
players.head()

In [ ]:
def output_players(players: pd.DataFrame, points_mode: str):
    # Load projections
    points_mode_str = points_mode.lower().replace(" ", "_")
    projs_path = os.path.join(DATA_DIR, "sleeper", str(year), f"projections_{points_mode_str}.csv")
    projs_raw = pd.read_csv(projs_path, index_col=[0])

    # Join and add columns
    players = players.join(projs_raw, how="left")
    players["adp"] = players["sum_weeks"].rank(ascending=False)  # TODO: real ADP?

    if not os.path.isdir(PROD_DIR):
        Path(PROD_DIR).mkdir(parents=True, exist_ok=True)
    players.to_csv(os.path.join(PROD_DIR, f"players_{points_mode_str}_v2.csv"), index=True)

In [ ]:
# Output production data
output_players(players, "PPR")
output_players(players, "Half PPR")